# Scraping: récupération de la page d'accueil de la RTBF

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [La RTBF](https://www.rtbf.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/saint-trond-anderlecht-le-sporting-veut-enchainer-bertaccini-de-retour-direct-audio-et-commente-18h30-11617902',
  'Direct audio et commenté, Saint-Trond – Anderlecht\xa0: 1-2, le but de Hazard annulé pour hos-jeu'],
 ['https://www.rtbf.be/article/braquage-au-louvre-le-musee-ferme-pour-la-journee-11618763',
  'Ce que l’on sait du cambriolage éclair au Louvre et du vol de bijoux napoléoniens à la valeur "inestimable"'],
 ['https://www.rtbf.be/article/absence-d-accord-pour-le-budget-federal-la-date-limite-naturelle-approche-selon-le-president-de-la-chambre-peter-de-roover-11618835',
  'Absence d\'accord pour le budget fédéral : la "date limite naturelle" approche, selon le président de la Chambre, Peter de Roover'],
 ['https://www.rtbf.be/article/f1-etats-unis-verstappen-veut-terminer-en-beaute-a-austin-direct-video-a-20h30-11618627',
  'F1 États-Unis\xa0: Verstappen veut terminer le week-end en beauté à Austin (Direct vidéo à 20h30)'],
 ['https://www.rtbf.

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/saint-trond-anderl...,"Direct audio et commenté, Saint-Trond – Anderl..."
1,https://www.rtbf.be/article/braquage-au-louvre...,Ce que l’on sait du cambriolage éclair au Louv...
2,https://www.rtbf.be/article/absence-d-accord-p...,"Absence d'accord pour le budget fédéral : la ""..."
3,https://www.rtbf.be/article/f1-etats-unis-vers...,F1 États-Unis : Verstappen veut terminer le we...
4,https://www.rtbf.be/article/l-il-de-moscou-pas...,L’œil de Moscou : pas de deux russo-américain ...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,La capsule de Jardins & loisirs
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le 13h
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Cultes et relais
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Master Crimes S03


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

OSError: Cannot save file into a non-existent directory: '..\data'

## Téléchargement d'un article et affichage du texte

In [ ]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

Mondiaux de gymnastique : que peut-on attendre de la Belgique sans Nina Derwael ?
Six gymnastes belges composent la délégation belge pour les championnats du monde de gymnastique artistique du 19 au 25 octobre à Jakarta en Indonésie. C’est le premier rendez-vous sans Nina Derwael qui a donc mis un terme à sa carrière cet été.
Les gymnastes évolueront individuellement, avec pour objectif de décrocher une place en finale du concours général ou en finales par agrès, chaque nation ne pouvant aligner que six gymnastes chez les messieurs et quatre chez les dames, au maximum, avec aussi un maximum de trois représentants par agrès.
Lisa Vaelen  (De Gympies Keerbergen) et  Jade Vansteenkiste  (Izegem) seront les porte-drapeaux de la Belgique côté féminin.
C’est la deuxième fois sa carrière que Lisa Vaelen participera aux mondiaux. L’Anversoise, 21 ans, avait fait ses débuts sur la scène mondiale il y a trois ans à Liverpool, en Angleterre, terminant sixième en finale du saut et 11e du concours 

### Nettoyage du texte à l'aide d'expressions régulières

In [ ]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Six gymnastes belges composent la délégation belge pour les championnats du monde de gymnastique artistique du 19 au 25 octobre à Jakarta en Indonésie. C’est le premier rendez-vous sans Nina Derwael qui a donc mis un terme à sa carrière cet été.
Les gymnastes évolueront individuellement, avec pour objectif de décrocher une place en finale du concours général ou en finales par agrès, chaque nation ne pouvant aligner que six gymnastes chez les messieurs et quatre chez les dames, au maximum, avec aussi un maximum de trois représentants par agrès.
Lisa Vaelen  (De Gympies Keerbergen) et  Jade Vansteenkiste  (Izegem) seront les porte-drapeaux de la Belgique côté féminin.
C’est la deuxième fois sa carrière que Lisa Vaelen participera aux mondiaux. L’Anversoise, 21 ans, avait fait ses débuts sur la scène mondiale il y a trois ans à Liverpool, en Angleterre, terminant sixième en finale du saut et 11e du concours général.
Son objectif sera encore de rejoindre une finale, elle qui s’alignera au 

### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/rtbf_example.txt'

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp

## Télécharger automatiquement des PDF

In [8]:
requests.get("https://max.de.wilde.web.ulb.be/camille/",verify=False)

d:\Cours_ULB\Traitement automatique de Corpus\tac\tac_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>